# FINM 25000
## Summer 2025
### Mark Hendricks

# Homework #4
**Background Case:** Grantham, Mayo, and Van Otterloo, 2012: Estimating the Equity Risk Premium [9-213-051]

**Sections 1 and 2 will...**
* NOT be graded.
* NOT be covered on the midterm.
* Rather, these sections will be discussed after the midterm.

# 1 GMO
*It requires no empirical analysis; answer solely based on the material given in the case.*

### 1. GMO's approach.

**(a) Why does GMO believe they can more easily predict long-run than short-run asset class performance?**

GMO believed that predicting long-run performance was easier than short-run performance since they thought that in the short-run, the stock market was a "voting machine" (built of hype and interest) whereas in the long-run, the stock market was a "weighing machine"  (built of fundamental equity value weightings).

**(b) What predicting variables does the case mention are used by GMO? Does this fit with the goal of long-run forecasts?**

The predicting variable used by GMO: dividend yield, percentage change in the PE multiple, percentage chage in profit margins, percentage change in sales per share.

These variables fit the goal of long-run forecasts. GMO uses a 7-year horizon for their forecasts.

**(c) How has this approach led to contrarian positions?**

This has led to contrarian positions because GMO believes that acutal market prices can deviate from fundamental value.

**(d) How does this approach raise business risk and managerial career risk?**

*(Your answer here)*

### 2. The market environment.

**(a) We often estimate the market risk premium by looking at a large sample of historic data. What reasons does the case give to be skeptical that the market risk premium will be as high in the future as it has been over the past 50 years?**

*(Your answer here)*

**(b) In 2007, GMO forecasts real excess equity returns will be negative. What are the biggest drivers of their pessimistic conditional forecast relative to the unconditional forecast. (See Exhibit 9.)**

*(Your answer here)*

**(c) In the 2011 forecast, what components has GMO revised most relative to 2007? Now how does their conditional forecast compare to the unconditional? (See Exhibit 10.)**

*(Your answer here)*

### 3. Consider the asset class forecasts in Exhibit 1.

**(a) Which asset class did GMO estimate to have a negative 10-year return over 2002-2011?**

*(Your answer here)*

**(b) Which asset classes substantially outperformed GMO's estimate over that time period?**

*(Your answer here)*

**(c) Which asset classes substantially underperformed GMO's estimate over that time period?**

*(Your answer here)*

### 4. Fund Performance.

**(a) In which asset class was GMWAX most heavily allocated throughout the majority of 1997-2011?**

*(Your answer here)*

**(b) Comment on the performance of GMWAX versus its benchmark. (No calculation needed; simply comment on the comparison in the exhibits.)**

*(Your answer here)*

# 2 Analyzing GMO

This section utilizes data in the file, `gmo_analysis_data.xlsx` on the `returns` tab.

Examine GMO's performance. For simplicity, just examine total returns.¹

In [474]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load the data from the 'returns' tab
file_path = './gmo_analysis_data.xlsx'
returns_df = pd.read_excel(
    file_path, 
    sheet_name='total returns', 
    index_col=0,
    parse_dates=[0])

returns_df.head()

,SPY,GMWAX,GMGEX
date,,,
1996-12-31,-0.023292,-0.022094,-0.013000
1997-01-31,0.061786,0.014735,0.034448
1997-02-28,0.009565,0.022265,0.012733
1997-03-31,-0.045721,-0.015152,-0.016441
1997-04-30,0.064368,-0.006731,0.000000


### 1. Calculate the mean, volatility, and Sharpe ratio for GMWAX. Do this for three samples:
* from inception through 2011
* 2012-present
* inception - present

Has the mean, vol, and Sharpe changed much since the case?

In [475]:
def calc_return_metrics(data, as_df=False, adj=12):
    """
    Calculate return metrics for a DataFrame of assets.

    Args:
        data (pd.DataFrame): DataFrame of asset returns.
        as_df (bool, optional): Return a DF or a dict. Defaults to False (return a dict).
        adj (int, optional): Annualization. Defaults to 12.

    Returns:
        Union[dict, DataFrame]: Dict or DataFrame of return metrics.
    """
    summary = dict()
    summary["Annualized Return"] = data.mean() * adj
    summary["Annualized Volatility"] = data.std() * np.sqrt(adj)
    summary["Annualized Sharpe Ratio"] = (
        summary["Annualized Return"] / summary["Annualized Volatility"]
    )
    return pd.DataFrame(summary, index=data.columns) if as_df else summary

# Inception to 2011
pre_2012 = returns_df[['GMWAX']].loc[:'2011']

summary_pre_2012 = calc_return_metrics(pre_2012, as_df=True)
summary_pre_2012.rename(index={'GMWAX': 'GMWAX (-2011)'}, inplace=True)

# 2012 to present
post_2011 = returns_df[['GMWAX']].loc['2012':]
summary_post_2011 = calc_return_metrics(post_2011, as_df=True)
summary_post_2011.rename(index={'GMWAX': 'GMWAX (2012-)'}, inplace=True)

# inception to present
summary_rets = calc_return_metrics(returns_df[['GMWAX']], as_df=True)
summary_rets.rename(index={'GMWAX': 'GMWAX (Full)'}, inplace=True)

display(pd.concat([summary_pre_2012, summary_post_2011, summary_rets]))

,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio
GMWAX (-2011),0.074759,0.110348,0.677488
GMWAX (2012-),0.061107,0.093605,0.652821
GMWAX (Full),0.068311,0.102652,0.665467


The mean annualized return has decreased slightly since prior to the case by about 1.3%. The vol and sharpe ratio have remained relatively steady. 

### 2. GMO believes a risk premium is compensation for a security’s tendency to lose money at “bad times”. For all three samples, analyze extreme scenarios by looking at
* Min return
* 5th percentile (VaR-5th)
* Maximum drawdown

In [489]:
# Your code here

def risk_metrics(returns_df, quantile):
  summary = pd.DataFrame()
  summary['Kurtosis'] = returns_df.kurtosis()
  summary['Var (0.05)'] = returns_df.quantile(q = 0.05, axis='index') #fifth quantile
  summary["Min"] = returns_df.min()
  summary["Max"] = returns_df.max()

  #max drawdown calculations
  index = 1000 * (1 + returns_df).cumprod() #cumulative product of all returns given $1000
  peaks = index.cummax() #highest points of the data at or before that date
  drawdowns = (index - peaks) / peaks #pct. calculation for drawdowns ()
  summary['Max Drawdown'] = drawdowns.min() #smallest on an absolute basis (more neg. the bigger drawdown)

  # dates for maximum / peak of each drawdown
  summary['Peak (in max. drawdown period)'] = peaks.idxmax()

  #dates for minimum / troughs of each drawdown
  summary['Trough (in max. drawdown period)'] = drawdowns.idxmin()

  recovery_dates = dict()
  for col in returns_df.columns:
      peak_date = summary.loc[col, 'Peak (in max. drawdown period)']
      trough_date = summary.loc[col, 'Trough (in max. drawdown period)']

      peak_value = index[col].loc[peak_date]

      index_after_trough = index[col].loc[trough_date:]

      recovery_date = index_after_trough[peak_value <= index_after_trough].index.min()
      recovery_dates[col] = recovery_date if recovery_date else pd.NaT

  summary['Recovery Date'] = recovery_dates

  return summary.T

risk_metrics_GMWAX = pd.DataFrame()
risk_metrics_GMWAX['-2012'] = risk_metrics(pre_2012, 0.05)
risk_metrics_GMWAX['2011-'] = risk_metrics(post_2011, 0.05)
risk_metrics_GMWAX['inception-present'] = risk_metrics(returns_df[['GMWAX']], 0.05)
display(risk_metrics_GMWAX)

,-2012,2011-,inception-present
Kurtosis,3.016184,2.128437,2.79974
Var (0.05),-0.043986,-0.036933,-0.040435
Min,-0.145129,-0.114967,-0.145129
Max,0.085656,0.078058,0.085656
Max Drawdown,-0.293614,-0.216795,-0.293614
Peak (in max. drawdown period),2011-04-29 00:00:00,2025-06-30 00:00:00,2025-06-30 00:00:00
Trough (in max. drawdown period),2009-02-27 00:00:00,2022-09-30 00:00:00,2009-02-27 00:00:00
Recovery Date,2011-04-29 00:00:00,2025-06-30 00:00:00,2025-06-30 00:00:00


**(a) Does GMWAX have high or low tail-risk as seen by these stats?**

The calculated excess kurtosis of GMWAX is 2.8 which suggests fatter tails and a higher likelihood of big downside and upside events. However, the VaR 0.05 is only 4.04% and the max drawdown is -30% which suggests that GMWAX has lower severity of those tails events.

**(b) Does that vary much across the two subsamples?**

Yes, the tail-risk varies signficantly between the two subsamples. The VaR is signficantly improved, the worth month return was less severe, the max drawdown improved by almost 8%, and the kurtosis was lower. As a result, GMWAX's tail risk was considerably lower in the 2012-present subsample comapred to the earlier period.

### 3. For all three samples, regress excess returns of GMWAX on excess returns of SPY.

In [477]:
# Your code here

**(a) Report the estimated alpha, beta, and r-squared.**

*(Your answer here)*

**(b) Is GMWAX a low-beta strategy? Has that changed since the case?**

*(Your answer here)*

**(c) Does GMWAX provide alpha? Has that changed across the subsamples?**

*(Your answer here)*

--- 
¹Technically, a Sharpe ratio is defined on *excess* returns, but the difference is negligible, and it is common to examine total returns.

# 3 Forecast Regressions

This section utilizes data in the file, `gmo_analysis_data.xlsx`, on the `signals` tab.

In [478]:
# Load the data from the 'signals' tab
file_path = './gmo_analysis_data.xlsx'
signals_df = pd.read_excel(
    file_path, 
    sheet_name='signals', 
    index_col=0,
    parse_dates=[0])

signals_df.head()

,SPX D/P,SPX E/P,T-Note 10YR
date,,,
1996-12-31,0.019651,0.050962,0.06418
1997-01-31,0.018455,0.048037,0.06494
1997-02-28,0.018502,0.047750,0.06552
1997-03-31,0.019427,0.054411,0.06903
1997-04-30,0.018430,0.051255,0.06718


### 1. Consider the lagged regression, where the regressor, $(X_t)$ is a period behind the target, $(r^{SPY}_{t+1})$.

$$r_{t+1}^{SPY} = \alpha^{SPY,X} + (\beta^{SPY,X})' X_{t} + \epsilon_{t+1}^{SPY,X} \quad (1)$$

Estimate (1) and report the $R^2$, as well as the OLS estimates for $\alpha$ and $\beta$. Do this for...

* **X** as a single regressor, the dividend-price ratio.
* **X** as a single regressor, the earnings-price ratio.
* **X** as three regressors, the dividend-price ratio, the earnings-price ratio, and the 10-year yield.

For each, report the r-squared.

In [479]:
def lagged_regression(Y, X, reg_name):
    """
    Calculates lagged regression of Y against X, handling both univariate and multivariate cases.

    Args:
        Y (pd.Series or pd.DataFrame): The dependent variable (e.g., SPY returns).
        X (pd.Series or pd.DataFrame): The independent variable(s) (e.g., D/P ratio, E/P ratio, or multiple signals).
        reg_name (str): The name to use for the regression result in the output DataFrame.

    Returns:
        pd.DataFrame: Summary of regression results (Alpha, Betas, R-Squared).
    """
    # Shift the explanatory variable down 1 & drop na rows
    X_lag = X.shift(1).dropna()

    # Align response indices with X_lag indices
    Y = Y.loc[X_lag.index]

    # set-up of Ordinary Least Sqaured Regression (drop missing values & add constant for regression)
    results = sm.OLS(Y, sm.add_constant(X_lag), missing = 'drop').fit()

    # constant and slope of explanatory variable (index 0 and 1 respectively)
    parameters = results.params

    intercept = parameters.iloc[0] # returns in excess of the market

    summary = dict()
    summary['Alpha'] = intercept
    summary['R-Squared'] = results.rsquared

    if isinstance(X, pd.Series):
        summary['Beta'] = parameters.iloc[1]
    else: # The multivariate case
        for col, beta_val in parameters.iloc[1:].items():
            summary[f'{col} Beta'] = beta_val

    return pd.DataFrame([summary], index=[reg_name])

# Let X be dividend-price ratio
spy = returns_df[['SPY']]

#Univariate regressions
forecast_dp_ratio = lagged_regression(spy, signals_df['SPX D/P'], 'SPX D/P')
print(forecast_dp_ratio.T)
print()
forecast_ep_ratio = lagged_regression(spy, signals_df['SPX E/P'], 'SPX E/P')
print(forecast_ep_ratio.T)
print()
# Multivariate regression
forecast_multi = lagged_regression(spy, signals_df[['SPX D/P', 'SPX E/P', 'T-Note 10YR']], 'Multivariate Regr')
print(forecast_multi.T)

            SPX D/P
Alpha     -0.009780
R-Squared  0.008636
Beta       1.024300

            SPX E/P
Alpha     -0.002856
R-Squared  0.003184
Beta       0.221994

                  Multivariate Regr
Alpha                     -0.003614
R-Squared                  0.010242
SPX D/P Beta               1.484509
SPX E/P Beta              -0.239207
T-Note 10YR Beta          -0.057215


### 2. For each of the three regressions, let's try to utilize the resulting forecast in a trading strategy.

* Build the forecasted SPY returns: $\hat{r}^{SPY}_{t+1}$. Note that this denotes the forecast made using $X_t$ to forecast the $(t+1)$ return.

* Set the scale of the investment in SPY equal to 100 times the forecasted value:
$$ w_t = 100 \cdot \hat{r}^{SPY}_{t+1} $$
We are not taking this scaling too seriously. We just want the strategy to go bigger in periods where the forecast is high and to withdraw in periods where the forecast is low, or even negative. Later, we'll reset the scaling to make sure it is all comparable.

* Calculate the return on this strategy:
$$ r^*_{t+1} = w_t r^{SPY}_{t+1} $$

You should now have the trading strategy returns, $r^*$ for each of the forecasts. For each strategy, estimate
* mean, volatility, Sharpe,
* max-drawdown
* 5th quantile of returns
* market alpha
* market beta
* market Information ratio

In [480]:
# Your code here

forecast_df = pd.DataFrame()

# Forecast returns for DP strat
alpha_dp = forecast_dp_ratio.loc['SPX D/P', 'Alpha']
beta_dp = forecast_dp_ratio.loc['SPX D/P', 'Beta']
forecast_df['Dividend-Price'] = alpha_dp + beta_dp * signals_df['SPX D/P']

# Forecast returns for EP strat
alpha_ep = forecast_ep_ratio.loc['SPX E/P', 'Alpha']
beta_ep = forecast_ep_ratio.loc['SPX E/P', 'Beta']
forecast_df['Earnings-Price'] = alpha_ep + beta_ep * signals_df['SPX E/P']

# Forecast returns for Multi Regr strat
alpha_multi = forecast_multi.loc['Multivariate Regr', 'Alpha']
beta_multi_dp = forecast_multi.loc['Multivariate Regr', 'SPX D/P Beta']
beta_multi_ep = forecast_multi.loc['Multivariate Regr', 'SPX E/P Beta']
beta_multi_US10Y = forecast_multi.loc['Multivariate Regr', 'T-Note 10YR Beta']
forecast_df['Multivariate Regr'] = alpha_multi + beta_multi_dp * signals_df['SPX D/P'] + \
                                                 beta_multi_ep * signals_df['SPX E/P'] + \
                                                 beta_multi_US10Y * signals_df['T-Note 10YR']

print("Forecasted returns:")
display(forecast_df)

# Calculate and siplay forecasted weights
forecast_wt = 100 * forecast_df[['Dividend-Price', 'Earnings-Price', 'Multivariate Regr']]

print()
print("Forecasted weights:")
display(forecast_wt)

# Calculate scaled returns
returns_scaled = forecast_wt.multiply(spy['SPY'], axis='index')
returns_scaled['SPY'] = spy
print('Returns scaled:')
display(returns_scaled)

Forecasted returns:


,Dividend-Price,Earnings-Price,Multivariate Regr
date,,,
1996-12-31,0.010349,0.008457,0.009696
1997-01-31,0.009124,0.007808,0.008576
1997-02-28,0.009172,0.007744,0.008682
1997-03-31,0.010119,0.009223,0.008261
1997-04-30,0.009098,0.008522,0.007641
...,...,...,...
2025-02-28,0.003233,0.005860,0.003445
2025-03-31,0.004117,0.006389,0.004158
2025-04-30,0.004270,0.006460,0.004329



Forecasted weights:


,Dividend-Price,Earnings-Price,Multivariate Regr
date,,,
1996-12-31,1.034862,0.845696,0.969557
1997-01-31,0.912356,0.780763,0.857629
1997-02-28,0.917170,0.774372,0.868174
1997-03-31,1.011918,0.922261,0.826052
1997-04-30,0.909795,0.852190,0.764135
...,...,...,...
2025-02-28,0.323281,0.586026,0.344505
2025-03-31,0.411678,0.638939,0.415769
2025-04-30,0.427042,0.646044,0.432864


Returns scaled:


,Dividend-Price,Earnings-Price,Multivariate Regr,SPY
date,,,,
1996-12-31,-0.024104,-0.019698,-0.022583,-0.023292
1997-01-31,0.056371,0.048241,0.052990,0.061786
1997-02-28,0.008773,0.007407,0.008304,0.009565
1997-03-31,-0.046266,-0.042167,-0.037768,-0.045721
1997-04-30,0.058562,0.054854,0.049186,0.064368
...,...,...,...,...
2025-02-28,-0.004104,-0.007439,-0.004373,-0.012695
2025-03-31,-0.022939,-0.035602,-0.023167,-0.055721
2025-04-30,-0.003703,-0.005601,-0.003753,-0.008670


In [481]:
def univariate_regression(funds, explanatory):
    """
    Function is designed to calculate the univariate regression of y against X.
    Can also do downside beta (when market < 0) and upside beta (when market > 0)

    Returns:
        DataFrame: Summary of results
    """
    reg_results = []
    for fund in funds.columns:
      response = funds[fund]
        # set-up of Ordinary Least Sqaured Regression (drop missing values & add constant for regression)
      results = sm.OLS(response, sm.add_constant(explanatory), missing = 'drop').fit()

      # constant and slope of explanatory variable (index 0 and 1 respectively)
      parameters = results.params

      intercept = parameters.iloc[0] # returns in excess of the market
      beta = parameters.iloc[1]

      summary = dict()

      summary['Alpha'] = intercept * 12
      summary['Beta'] = beta

      residuals = results.resid

      #returns in excess of the market penalized by the variance of the regression
      summary['Information Ratio'] = (intercept / residuals.std()) * np.sqrt(12) #annualize the ratio

      reg_results.append(pd.DataFrame(summary, index = [response.name]))
    return pd.concat(reg_results)

return_metrics_df = calc_return_metrics(returns_scaled, as_df=True).T

risk_metrics_df = risk_metrics(returns_scaled, 0.05)

univ_regr_df = univariate_regression(returns_scaled, spy).T

combined_df = pd.concat([return_metrics_df, risk_metrics_df, univ_regr_df])
display(combined_df)

,Dividend-Price,Earnings-Price,Multivariate Regr,SPY
Annualized Return,0.063087,0.068489,0.075874,0.103165
Annualized Volatility,0.177818,0.150343,0.178715,0.154125
Annualized Sharpe Ratio,0.354784,0.455553,0.424552,0.669357
Kurtosis,12.807675,5.095907,11.477248,0.749984
Var (0.05),-0.070084,-0.069699,-0.070814,-0.074563
Min,-0.338421,-0.244285,-0.31348,-0.165187
Max,0.235171,0.143509,0.2428,0.126983
Max Drawdown,-0.785262,-0.659486,-0.758419,-0.507976
Peak (in max. drawdown period),2025-06-30 00:00:00,2025-06-30 00:00:00,2025-06-30 00:00:00,2025-06-30 00:00:00
Trough (in max. drawdown period),2009-02-27 00:00:00,2009-02-27 00:00:00,2009-02-27 00:00:00,2009-02-27 00:00:00


### 3. The GMO case mentions that stocks under-performed short-term bonds from 2000-2011. Does the dynamic portfolio above under-perform the risk-free rate over this time?

In [482]:
# Your code here
file_path = './gmo_analysis_data.xlsx'
risk_free_rate_df = pd.read_excel(
    file_path, 
    sheet_name='risk-free rate', 
    index_col=0,
    parse_dates=[0])

risk_free_rate_df['TBill 3M'] = risk_free_rate_df['TBill 3M'] / 12 #de annualize the risk free rate
display(risk_free_rate_df.head(5))

returns_scaled['Risk-Free Rate'] = risk_free_rate_df['TBill 3M']

returns_scaled_subsample = returns_scaled.loc['2000':'2011']

display(returns_scaled_subsample)

returns_metrics_df2 = calc_return_metrics(returns_scaled_subsample, as_df=True).T
risk_metrics_df2 = risk_metrics(returns_scaled_subsample, 0.05)
univ_regr_df2 = univariate_regression(returns_scaled, spy).T
combined_df2 = pd.concat([returns_metrics_df2, risk_metrics_df2, univ_regr_df2])
display(combined_df2)

,TBill 3M
date,
1996-12-31,0.004309
1997-01-31,0.004289
1997-02-28,0.004350
1997-03-31,0.004435
1997-04-30,0.004361


,Dividend-Price,Earnings-Price,Multivariate Regr,SPY,Risk-Free Rate
date,,,,,
2000-01-31,-0.010792,-0.024905,-0.007069,-0.049787,0.004743
2000-02-29,-0.003756,-0.007888,-0.002752,-0.015226,0.004817
2000-03-31,0.014035,0.047120,0.008780,0.096895,0.004893
2000-04-28,-0.005889,-0.018054,-0.002876,-0.035121,0.004857
2000-05-31,-0.003012,-0.008956,-0.000836,-0.015723,0.004683
...,...,...,...,...,...
2011-08-31,-0.065626,-0.073686,-0.049852,-0.054976,0.000008
2011-09-30,-0.095844,-0.101922,-0.073432,-0.069449,0.000017
2011-10-31,0.125150,0.141577,0.097292,0.109147,-0.000013


,Dividend-Price,Earnings-Price,Multivariate Regr,SPY,Risk-Free Rate
Annualized Return,-0.002487,0.003672,0.0033,0.018072,0.022875
Annualized Volatility,0.227954,0.181086,0.21948,0.162266,0.005782
Annualized Sharpe Ratio,-0.01091,0.020276,0.015034,0.111374,3.956389
Kurtosis,9.807471,4.550147,10.10738,0.618922,-1.099445
Var (0.05),-0.094131,-0.092863,-0.075061,-0.081276,0.000034
Min,-0.338421,-0.244285,-0.31348,-0.165187,-0.000013
Max,0.235171,0.143509,0.2428,0.109147,0.005324
Max Drawdown,-0.785262,-0.659486,-0.758419,-0.507976,-0.000013
Peak (in max. drawdown period),2007-10-31 00:00:00,2007-10-31 00:00:00,2007-10-31 00:00:00,2007-10-31 00:00:00,2011-09-30 00:00:00
Trough (in max. drawdown period),2009-02-27 00:00:00,2009-02-27 00:00:00,2009-02-27 00:00:00,2009-02-27 00:00:00,2011-10-31 00:00:00


Based on the above data, GMO is clearly correct when saying that short-term bonds outperformed stocks. 3-month treasuries yielded 2.28% while SPY yield 1.8% over that time period. Additionally, all three dynamic portfolios under-performed the Risk-Free rate. There annualized mean returns were all signficantly lower with the dividend-price strat even yielding negative returns.

The statistics make sense when considering the overall macroeconomic factors including the 2008 financial crisis.

### 4. Based on the regression estimates, in how many periods do we estimate a negative risk premium? That is, in how many periods is our forecasted excess return negative?

In [483]:
# Your code here

forecasted_excess_returns = forecast_df.sub(risk_free_rate_df['TBill 3M'], axis=0)
negative_count = (forecasted_excess_returns < 0).sum()
print(negative_count)

Dividend-Price       42
Earnings-Price        1
Multivariate Regr    40
dtype: int64


### 5. Do you believe increased risk is behind the out-performance of $\tilde{r}^*$ and $\tilde{r}^{gmo}$?

In [484]:
# Your code here
GMWAX = returns_df[['GMWAX']]
GMGEX = returns_df[['GMGEX']]

returns_metrics_df3 = calc_return_metrics(GMGEX, as_df=True).T
risk_metrics_df3 = risk_metrics(GMGEX, 0.05)
univ_regr_df3 = univariate_regression(GMGEX, returns_df[['SPY']]).T
combined_df3 = pd.concat([returns_metrics_df3, risk_metrics_df3, univ_regr_df3])
display(combined_df3)

,GMGEX
Annualized Return,0.023802
Annualized Volatility,0.191033
Annualized Sharpe Ratio,0.124598
Kurtosis,59.543001
Var (0.05),-0.075489
Min,-0.658652
Max,0.124727
Max Drawdown,-0.761812
Peak (in max. drawdown period),2007-10-31 00:00:00
Trough (in max. drawdown period),2016-11-30 00:00:00


For $\tilde{r}^*$ no. All three strategies underperformed relative to SPY leading to lower mean returns, negative alphas, and negative info ratios. Additionally, all three strats experienced higher vols and larger maximum drawdowns. Lastly, all three had betas very close to 1. Thus, the strats did not demonstrate outperformance. They all generally provided lower risk-adjusted returns and higher vols and drawdowns. As a result, the increased did not translate to increased performance.

For $\tilde{r}^{gmo}$ no. GMGEX also underperformed relative to SPY. Lower mean returns, higher vol, lower sharpe, severe max drawdowns, and a negative alpha all show that on a risk-adjusted basis, increased risk is not behind the out-performance of GMO. 

# 4 Extensions
**This section is not graded, and you do not need to submit your answers. This is only provided for those interested in further study.**

This section utilizes data in the file, `gmo_analysis_data.xlsx`.

Reconsider the problem above, of estimating (1) for $x$. The reported $R^2$ was the in-sample $R^2$–it examined how well the forecasts fit in the sample from which the parameters were estimated. Let's consider the out-of-sample r-squared. To do so, we need to do the following:

* Start at $t=60$.

* Estimate (1) only using data through time $t$.

* Use the estimated parameters of (1), along with $x_{t}$ to calculate the out-of-sample forecast for the following period, $t+1$.
$$ \hat{r}^{SPY}_{t+1} = \alpha^{t, SPY,x} + (\beta^{t, SPY,x})' x_t $$

* Calculate the $t+1$ forecast error,
$$ e^x_{t+1} = r^{SPY}_{t+1} - \hat{r}^{SPY}_{t+1} $$

* Move to $t=61$, and loop through the rest of the sample.

You now have the time-series of out-of-sample prediction errors, $e^x$.

Calculate the time-series of out-of-sample prediction errors $e^0$, which are based on the null forecast:
$$ \bar{r}^{SPY}_{t+1} = \frac{1}{t} \sum_{i=1}^{t} R^{SPY}_i $$
$$ e^0_{t+1} = r^{SPY}_{t+1} - \bar{r}^{SPY}_{t+1} $$


### 1. Report the out-of-sample $R^2$ :
$$ R^2_{OOS} = 1 - \frac{\sum_{t=61}^{T} (e^x_t)^2}{\sum_{t=61}^{T} (e^0_t)^2} $$

Note that unlike an in-sample r-squared, the out-of-sample r-squared can be anywhere between $(-\infty, 1]$.

Did this forecasting strategy produce a positive OOS r-squared?

In [485]:
# Your code here

### 2. Re-do problem 3.2 using this OOS forecast.
How much better/worse is the OOS Earnings-Price ratio strategy compared to the in-sample version of 3.2?

In [486]:
# Your code here